In [ ]:
import os
import json
from tkinter import Tk, Label, Button
from PIL import Image, ImageTk, UnidentifiedImageError

class ImageLabeler:
    def __init__(self, image_folder, output_file="labels.json"):
        self.image_folder = image_folder
        self.output_file = output_file
        self.images = sorted([f for f in os.listdir(image_folder)
                              if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))])
        self.labels = self.load_labels()
        self.labeled_history = []  # Track labeling order
        self.index = 0

        self.root = Tk()
        self.root.title("Image Labeler: D = Damaged | A = Not Damaged | R = Revert")

        self.label = Label(self.root)
        self.label.pack()

        self.btn_damaged = Button(self.root, text="Damaged (D)", command=lambda: self.label_image("damaged"))
        self.btn_damaged.pack(side="left", padx=10, pady=10)

        self.btn_not_damaged = Button(self.root, text="Not Damaged (A)", command=lambda: self.label_image("not_damaged"))
        self.btn_not_damaged.pack(side="left", padx=10, pady=10)

        self.btn_revert = Button(self.root, text="Revert (R)", command=self.revert_last)
        self.btn_revert.pack(side="right", padx=10, pady=10)

        self.root.bind("<d>", lambda event: self.label_image("damaged"))
        self.root.bind("<a>", lambda event: self.label_image("not_damaged"))
        self.root.bind("<r>", lambda event: self.revert_last())

        self.show_image()

        self.root.mainloop()

    def load_labels(self):
        if os.path.exists(self.output_file):
            with open(self.output_file, "r") as f:
                return json.load(f)
        return {}

    def save_labels(self):
        with open(self.output_file, "w") as f:
            json.dump(self.labels, f, indent=2)

    def show_image(self):
        while self.index < len(self.images):
            filename = self.images[self.index]
            if filename in self.labels:
                self.index += 1
                continue

            image_path = os.path.join(self.image_folder, filename)

            try:
                img = Image.open(image_path)
                img.thumbnail((800, 600))
                tk_img = ImageTk.PhotoImage(img)
                self.label.config(image=tk_img)
                self.label.image = tk_img
                return
            except (UnidentifiedImageError, OSError) as e:
                print(f"Error loading image {filename}: {e}")
                self.index += 1

        self.label.config(text="All images labeled or failed to load.")
        self.label.image = None

    def label_image(self, label):
        if self.index >= len(self.images):
            return
        filename = self.images[self.index]
        self.labels[filename] = label
        self.labeled_history.append(filename)
        self.index += 1
        self.save_labels()
        self.show_image()

    def revert_last(self):
        if not self.labeled_history:
            print("Nothing to revert.")
            return

        last_file = self.labeled_history.pop()
        if last_file in self.labels:
            del self.labels[last_file]
            self.index = self.images.index(last_file)
            self.save_labels()
            self.show_image()
            print(f"Reverted label for {last_file}")

# Usage
if __name__ == "__main__":
    image_folder = "/Volumes/Extreme SSD/MosquitoWingImages_processed/Culicidae_images_morph"  # <- Dein Pfad
    ImageLabeler(image_folder)


Error loading image ._culex_aida_f_l_06ce1fec12cf.png: cannot identify image file '/Volumes/Extreme SSD/MosquitoWingImages_processed/Culicidae_images_morph/._culex_aida_f_l_06ce1fec12cf.png'
Error loading image ._culex_aida_f_l_23df254b077b.png: cannot identify image file '/Volumes/Extreme SSD/MosquitoWingImages_processed/Culicidae_images_morph/._culex_aida_f_l_23df254b077b.png'
Error loading image ._culex_aida_f_l_89cbcf5cac7c.png: cannot identify image file '/Volumes/Extreme SSD/MosquitoWingImages_processed/Culicidae_images_morph/._culex_aida_f_l_89cbcf5cac7c.png'
Error loading image ._culex_aida_f_r_0feb4eb45b65.png: cannot identify image file '/Volumes/Extreme SSD/MosquitoWingImages_processed/Culicidae_images_morph/._culex_aida_f_r_0feb4eb45b65.png'
Error loading image ._culex_aida_f_r_d658252d976e.png: cannot identify image file '/Volumes/Extreme SSD/MosquitoWingImages_processed/Culicidae_images_morph/._culex_aida_f_r_d658252d976e.png'
Error loading image ._culex_aida_f_r_db8b658f